<a href="https://colab.research.google.com/github/omanofx/entregable_1/blob/Proyecto_final_Omar_Fernandez/cross_validation_Omar_Fernandez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aplicamos validación cruzada al dataset de hoteles ya procesado y depurado. Contiene todas las nuevas características creadas en el desafio de la clase 45.

## Librerías

In [1]:
import numpy as np
import pandas as pd

# memory management
import gc

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

import xgboost as xgb

from sklearn.metrics import mean_squared_error

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, HalvingRandomSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error, r2_score

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

## Funciones propias

In [2]:
def tiene_espacios_en_blanco(columna: str):
    return any(x.isspace() for x in columna)

In [3]:
# Definir la función analizar_dataframe
def analizar_dataframe(df1: pd.DataFrame, porcentaje_tolerancia: float):
    '''
    ## Esta función se utiliza para comprobar la consistencia de los datos de un DataFrame
    ### evita invocar a .info(), .isnull()
    #### Porcentaje de tolerancia: valor porcentual que se tolera para los valores nulos de
    #### cada caracteristica del dataset. Si el valor se encuentra por sobre la tolerancia,
    #### se indica como 'BORRAR' la caracteristica.
    '''
    # Validar que porcentaje_tolerancia esté entre 0 y 100
    if not (0 <= porcentaje_tolerancia <= 100):
        print("Error: El porcentaje de tolerancia debe estar entre 0 y 100.")
        return

    porcentaje_perdidos = df1.isnull().sum() * 100 / len(df1)
    total_nulos = df1.isnull().sum()
    tipo_dato = df1.dtypes
    valores_no_nulos_por_columna = df1.count()
    descripcion = df1.describe().transpose()

    resultado_analisis = pd.DataFrame({
        'Tipo de dato': tipo_dato,
        'Total No nulos': valores_no_nulos_por_columna,
        'Total nulos': total_nulos,
        'Porcentaje Nulos': round(porcentaje_perdidos, 2),
        'Borrar': np.where(porcentaje_perdidos > porcentaje_tolerancia, 'BORRAR', ''),
        'Columna con Espacios': df1.columns.to_series().apply(lambda x: tiene_espacios_en_blanco(x))
    })

    # Agregar las columnas de la descripción al resultado_analisis
    resultado_analisis = pd.concat([resultado_analisis, round(descripcion, 2)], axis=1)

    estilo_resultado = (
        resultado_analisis.style
        .applymap(lambda x: 'background-color: red',
                  subset=pd.IndexSlice[resultado_analisis['Porcentaje Nulos'] > porcentaje_tolerancia, 'Porcentaje Nulos'])
        .background_gradient(cmap='Reds', subset=['Porcentaje Nulos'])
        .applymap(lambda x: 'background-color: red' if x else '',
                  subset=pd.IndexSlice[resultado_analisis['Columna con Espacios'], 'Columna con Espacios'])
        .background_gradient(cmap='Reds', subset=['Columna con Espacios'])
    )


    return estilo_resultado

In [4]:
def calcular_precision_modelo(y_test, y_pred):
  '''
  ## Calcula la precisión del modelo
  '''
  accuracy_puntaje = accuracy_score(y_test, y_pred)

  print("Precisión del modelo: {:.2f}%".format(accuracy_puntaje * 100))

  if accuracy_puntaje > 0.9:
      print("El modelo tiene un alto rendimiento en la clasificación.")
  else:
      print("El modelo podría necesitar mejoras para lograr un rendimiento más alto.")

## Carga del Data set

El dataset ya fue procesado, por lo cual no contiene valores perdidos. Además, contiene las nuevas caraterísticas que fueron añadidas en previos desafios.

In [ ]:
#dataset procesado: última version 12-2-2024
#df_reservas = pd.read_csv("https://raw.githubusercontent.com/omanofx/entregable_1/Proyecto_final_Omar_Fernandez/df_reservas_ya_procesado.csv", sep = ",")


In [7]:
df_reservas_numerico = pd.read_csv("reservas_foward.csv")

In [8]:
# Deja unicamente las características númericas ya que las categoricas fueron convertirdas con Label Enconder.
df_reservas_numerico = df_reservas.select_dtypes(include='number').sample(5000) # limito el DF

In [9]:
analizar_dataframe(df_reservas_numerico, porcentaje_tolerancia=0)

,Tipo de dato,Total No nulos,Total nulos,Porcentaje Nulos,Borrar,Columna con Espacios,count,mean,std,min,25%,50%,75%,max
total_pasajeros,int64,5000,0,0.000000,,False,5000.000000,2.000000,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000
required_car_parking_spaces,int64,5000,0,0.000000,,False,5000.000000,0.090000,0.290000,0.000000,0.000000,0.000000,0.000000,1.000000
market_segment_le,int64,5000,0,0.000000,,False,5000.000000,3.150000,1.250000,0.000000,3.000000,4.000000,4.000000,4.000000
country_le,int64,5000,0,0.000000,,False,5000.000000,86.690000,44.960000,1.000000,51.000000,76.000000,135.000000,174.000000
total_of_special_requests,int64,5000,0,0.000000,,False,5000.000000,0.620000,0.710000,0.000000,0.000000,0.000000,1.000000,2.000000
lead_time,int64,5000,0,0.000000,,False,5000.000000,71.980000,73.810000,0.000000,10.000000,46.000000,117.000000,297.000000
adr,float64,5000,0,0.000000,,False,5000.000000,103.550000,41.450000,54.890000,68.700000,95.000000,129.970000,227.100000
booking_changes,int64,5000,0,0.000000,,False,5000.000000,0.200000,0.490000,0.000000,0.000000,0.000000,0.000000,2.000000
arrival_date_year,int64,5000,0,0.000000,,False,5000.000000,2016.220000,0.690000,2015.000000,2016.000000,2016.000000,2017.000000,2017.000000
distribution_channel_le,int64,5000,0,0.000000,,False,5000.000000,2.570000,0.900000,0.000000,3.000000,3.000000,3.000000,3.000000


In [11]:

X = df_reservas_numerico.drop('is_canceled', axis=1)
y = df_reservas_numerico['is_canceled']

# Separación train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model= xgb.XGBClassifier(learning_rate=0.001)

# Ajustar modelo
model.fit(X_train, y_train)

# Predicciones
predicciones = model.predict(X_test)

# Llama a la función para calcular la precisión del modelo
calcular_precision_modelo(y_test, predicciones )


Precisión del modelo: 72.60%
El modelo podría necesitar mejoras para lograr un rendimiento más alto.


In [12]:
# Lista de hiperparametros
params_1 = {'criterion': 'gini', 'splitter': 'best', 'max_depth': 5}
params_2 = {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 8}
params_3 = {'criterion': 'gini', 'splitter': 'random', 'max_depth': 10}

In [13]:
# Modelo 1
model.set_params(**params_1).fit(X_train, y_train)
print(f'Accuracy para Modelo 1 = {round(accuracy_score(y_test, model.predict(X_test)), 5)}')
# Modelo 2
model.set_params(**params_2).fit(X_train, y_train)
print(f'Accuracy para Modelo 2 = {round(accuracy_score(y_test, model.predict(X_test)), 5)}')
# Modelo 3
model.set_params(**params_3).fit(X_train, y_train)
print(f'Accuracy para Modelo 3 = {round(accuracy_score(y_test, model.predict(X_test)), 5)}')

Accuracy para Modelo 1 = 0.726
Accuracy para Modelo 2 = 0.726
Accuracy para Modelo 3 = 0.726


In [14]:
params_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [5,6,7],
        'criterion':['entropy','gini']
        }

In [15]:
# Función para buscar los hiperparametros
def apply_search_cv(model, params_grid, X_train, y_train, X_test, y_test, search_method, cv_method, scoring="accuracy", **kwargs):
    grid_cv = search_method(model, params_grid, scoring=scoring, cv=cv_method, **kwargs)
    grid_cv.fit(X_train, y_train)

    print("Mejores parametros:", grid_cv.best_params_)
    print("Mejor score de CV:", grid_cv.best_score_)
    print(f'Accuracy del modelo: {round(accuracy_score(y_test, grid_cv.predict(X_test)), 5)}')


In [16]:
# Grid Search
apply_search_cv(model, params_grid, X_train, y_train, X_test, y_test, search_method=GridSearchCV, cv_method=3)
print()
apply_search_cv(model, params_grid, X_train, y_train, X_test, y_test, search_method=GridSearchCV, cv_method=5)

Mejores parametros: {'colsample_bytree': 0.6, 'criterion': 'entropy', 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.6}
Mejor score de CV: 0.7200001175846352
Accuracy del modelo: 0.726

Mejores parametros: {'colsample_bytree': 0.6, 'criterion': 'entropy', 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.6}
Mejor score de CV: 0.72
Accuracy del modelo: 0.726


In [19]:
# Randomized Search CV
apply_search_cv(model, params_grid, X_train, y_train, X_test, y_test, search_method=RandomizedSearchCV, cv_method=5)
print()
apply_search_cv(model, params_grid, X_train, y_train, X_test, y_test, search_method=RandomizedSearchCV, cv_method=10)

Mejores parametros: {'subsample': 0.8, 'min_child_weight': 10, 'max_depth': 5, 'gamma': 0.5, 'criterion': 'gini', 'colsample_bytree': 1.0}
Mejor score de CV: 0.72
Accuracy del modelo: 0.726

Mejores parametros: {'subsample': 0.8, 'min_child_weight': 1, 'max_depth': 7, 'gamma': 0.5, 'criterion': 'gini', 'colsample_bytree': 0.8}
Mejor score de CV: 0.72
Accuracy del modelo: 0.726


In [20]:
# Halving GridSearch CV
apply_search_cv(model, params_grid, X_train, y_train, X_test, y_test, search_method=HalvingGridSearchCV, cv_method=5 ,factor=3)

Mejores parametros: {'colsample_bytree': 0.6, 'criterion': 'gini', 'gamma': 1.5, 'max_depth': 6, 'min_child_weight': 5, 'subsample': 1.0}
Mejor score de CV: 0.7197530864197532
Accuracy del modelo: 0.726


In [21]:
# Halving Randomized Search
apply_search_cv(model, params_grid, X_train, y_train, X_test, y_test, search_method=HalvingRandomSearchCV, cv_method=8, factor=3)

Mejores parametros: {'subsample': 1.0, 'min_child_weight': 1, 'max_depth': 5, 'gamma': 1, 'criterion': 'gini', 'colsample_bytree': 0.8}
Mejor score de CV: 0.7221792225662194
Accuracy del modelo: 0.726
